# News Article Scraping

In [1]:
from bs4 import BeautifulSoup

In [2]:
import urllib.request,sys,time
import requests
import pandas as pd
import numpy as np

## Loading the page

In [3]:
url = 'https://www.theverge.com/sitemaps/entries/2021/3'
page = requests.get(url)

In [4]:
page.status_code

200

In [5]:
page.text

'<?xml version="1.0" encoding="UTF-8"?>\n<urlset xmlns="http://www.sitemaps.org/schemas/sitemap/0.9">\n  <url>\n    <loc>https://www.theverge.com/21250695/best-laptops</loc>\n    <lastmod>2021-03-04T23:03:15+00:00</lastmod>\n  </url>\n  <url>\n    <loc>https://www.theverge.com/21266366/windows-10-pause-updates-install</loc>\n    <lastmod>2021-03-12T15:30:19+00:00</lastmod>\n  </url>\n  <url>\n    <loc>https://www.theverge.com/21280354/best-ipad-deals-apple</loc>\n    <lastmod>2021-03-01T14:28:15+00:00</lastmod>\n  </url>\n  <url>\n    <loc>https://www.theverge.com/21289209/best-apple-watch-deals</loc>\n    <lastmod>2021-03-01T18:19:19+00:00</lastmod>\n  </url>\n  <url>\n    <loc>https://www.theverge.com/21309820/best-wireless-earbuds</loc>\n    <lastmod>2021-03-12T13:00:29+00:00</lastmod>\n  </url>\n  <url>\n    <loc>https://www.theverge.com/21437954/xbox-series-x-s-preorder-us-retailers-microsoft-release-date</loc>\n    <lastmod>2021-03-05T16:37:48+00:00</lastmod>\n  </url>\n  <url>\n

## Parsing HTML data

In [6]:
soup = BeautifulSoup(page.text, "html.parser")

## Finding all articles during March 11-12

In [7]:
links = soup.find_all('url')

In [9]:
links

[<url>
 <loc>https://www.theverge.com/21250695/best-laptops</loc>
 <lastmod>2021-03-04T23:03:15+00:00</lastmod>
 </url>,
 <url>
 <loc>https://www.theverge.com/21266366/windows-10-pause-updates-install</loc>
 <lastmod>2021-03-12T15:30:19+00:00</lastmod>
 </url>,
 <url>
 <loc>https://www.theverge.com/21280354/best-ipad-deals-apple</loc>
 <lastmod>2021-03-01T14:28:15+00:00</lastmod>
 </url>,
 <url>
 <loc>https://www.theverge.com/21289209/best-apple-watch-deals</loc>
 <lastmod>2021-03-01T18:19:19+00:00</lastmod>
 </url>,
 <url>
 <loc>https://www.theverge.com/21309820/best-wireless-earbuds</loc>
 <lastmod>2021-03-12T13:00:29+00:00</lastmod>
 </url>,
 <url>
 <loc>https://www.theverge.com/21437954/xbox-series-x-s-preorder-us-retailers-microsoft-release-date</loc>
 <lastmod>2021-03-05T16:37:48+00:00</lastmod>
 </url>,
 <url>
 <loc>https://www.theverge.com/2020/9/17/21440300/ps5-playstation-5-preorder-price-release-date-buy</loc>
 <lastmod>2021-03-05T17:02:54+00:00</lastmod>
 </url>,
 <url>
 <l

Iterating through each page and gathering info

In [10]:
articles = []
for i in links:
    date = i.find('lastmod').text.strip()
    if("2021-03-11" in date or "2021-03-12" in date):
        url = i.find('loc').text.strip()
        page = requests.get(url)
        soup = BeautifulSoup(page.text, "html.parser")
        date = soup.find('time',attrs={'class':'c-byline__item'}).text.strip()
        title = soup.find('h1',attrs={'class':'c-page-title'}).text.strip()
        article = soup.find('div',attrs={'class':'c-entry-content'}).text.strip()
        author = soup.find('span',attrs={'class':'c-byline__author-name'}).text.strip()
        articles.append([date,title,article,author])

Load data onto DataFrame

In [11]:
df = pd.DataFrame(articles, columns=['Date','Title','Article','Author'])
df.head()

,Date,Title,Article,Author
0,"Mar 12, 2021, 10:30am EST",Windows 10 basics: How to pause and disable up...,"Windows 10 updates, which Microsoft sends to y...",Aliya Chaudhry
1,"Mar 12, 2021, 8:00am EST",The best wireless earbuds to buy right now,The true wireless earbuds market has never bee...,Chris Welch
2,"Mar 12, 2021, 10:00am EST",Stream it yourself,"PmsProxy, a partnered Twitch streamer who has ...",Bijan Stephen
3,"Mar 12, 2021, 9:00am EST","When games are hard on their hands, some playe...",As both a software engineer and an avid player...,Kait Sanchez
4,"Mar 11, 2021, 11:00am EST",Grassroots online efforts are forming a new qu...,"Almost exactly one year ago, Theo Hendrie thou...",Emma Banks


Save to JSON file

In [12]:
out = df.to_json("news.json",orient='records')
out